# pandas で concat する速度の比較


1. 行数が 1_000_000 くらいのテーブルを作成する
2. 1.で作成したテーブルを for 文使って行方向に 20 個結合する時間について，以下のケースで 100 回ずつ測定
   - list[pd.DataFrame] を作り pd.concat する
   - pd.DataFrame に毎回 pd.concat する
3. 2.を polars についても同様に試す


---


## import


In [1]:
import gc
import sys
from time import perf_counter

import numpy as np
import pandas as pd
import polars as pl
from tqdm import tqdm

In [2]:
sys.version

'3.11.4 (main, Jul 12 2023, 00:30:27) [GCC 11.3.0]'

In [3]:
pd.__version__

'2.1.1'

In [4]:
pl.__version__

'0.19.5'

## fn


In [5]:
def get_process_time_intervals(
    concat_mode: str = "many_times",
    lib_name: str = "pandas",
    num_try: int = 100,
    num_concat: int = 20,
    data_range: int = 1_000_000,
) -> list[float]:
    assert concat_mode in ["many_times", "only_once"], ValueError(
        f"concat_mode={concat_mode} は 'many_times' もしくは 'only_once' のみ使用可能です"
    )
    assert lib_name in ["pandas", "polars"], ValueError(
        f"lib_name={lib_name} は 'pandas' もしくは 'polars' のみ使用可能です"
    )

    x = np.arange(0, data_range, 1, dtype=np.float64)
    if lib_name == "pandas":
        df_tmp = pd.DataFrame(
            data={"col_0": x, "col_1": x, "col_2": x, "col_3": x, "col_4": x}
        )
    elif lib_name == "polars":
        df_tmp = pl.DataFrame(
            data={"col_0": x, "col_1": x, "col_2": x, "col_3": x, "col_4": x}
        )

    list_time: list[float] = []
    for _ in tqdm(range(num_try)):
        t_start = perf_counter()

        if concat_mode == "many_times":
            if lib_name == "pandas":
                df = pd.DataFrame()
                for _ in range(num_concat):
                    df = pd.concat(objs=[df, df_tmp], axis=0)
                # df.reset_index(drop=True, inplace=True)

            elif lib_name == "polars":
                df = pl.DataFrame()
                for _ in range(num_concat):
                    df = pl.concat(items=[df, df_tmp], how="vertical")

        elif concat_mode == "only_once":
            list_dfs: list = []
            for _ in range(num_concat):
                list_dfs.append(df_tmp)

            if lib_name == "pandas":
                df = pd.concat(objs=list_dfs, axis=0)
                # df.reset_index(drop=True, inplace=True)

            elif lib_name == "polars":
                df = pl.concat(items=list_dfs, how="vertical")

        t_elapsed = perf_counter() - t_start
        assert df.__len__() == num_concat * data_range, ValueError
        list_time.append(t_elapsed)

    del df
    gc.collect()

    return list_time

In [6]:
df_time = pd.DataFrame(
    data={
        "pandas_concat_list_dfs_only_once": get_process_time_intervals(
            concat_mode="only_once", lib_name="pandas"
        ),
        "pandas_concat_df_many_times": get_process_time_intervals(
            concat_mode="many_times", lib_name="pandas"
        ),
        "polars_concat_list_dfs_only_once": get_process_time_intervals(
            concat_mode="only_once", lib_name="polars"
        ),
        "polars_concat_df_many_times": get_process_time_intervals(
            concat_mode="many_times", lib_name="polars"
        ),
    }
)
df_time

100%|██████████| 100/100 [03:18<00:00,  1.99s/it]


,pandas_concat_list_dfs_only_once,pandas_concat_df_many_times,polars_concat_list_dfs_only_once,polars_concat_df_many_times
0,0.577719,2.408093,0.166258,1.950960
1,0.481048,3.251420,0.170176,2.380181
2,0.230936,2.797252,0.176868,2.278071
3,0.274883,2.510188,0.165578,2.302411
4,0.262519,2.890913,0.172486,2.342867
...,...,...,...,...
95,0.178446,2.647607,0.149601,2.036742
96,0.184727,2.141587,0.139801,2.038443
97,0.181073,2.114850,0.146241,1.946197
98,0.185886,2.061302,0.166531,1.908553


In [7]:
df_desc = df_time.describe().T
df_desc.drop(columns=["count"], inplace=True)
df_desc["median"] = df_time.median()
df_desc.sort_values(by="median", inplace=True)

df_desc.style.bar(color="blue", align="zero")

,mean,std,min,25%,50%,75%,max,median
polars_concat_list_dfs_only_once,0.176083,0.087882,0.131148,0.142313,0.153726,0.163281,0.706469,0.153726
pandas_concat_list_dfs_only_once,0.235592,0.100332,0.160950,0.181542,0.195120,0.235982,0.620031,0.195120
polars_concat_df_many_times,1.988170,0.169783,1.736401,1.881851,1.963711,2.038146,2.908779,1.963711
pandas_concat_df_many_times,2.629965,0.559238,1.813775,2.191198,2.526360,2.869135,4.442604,2.526360
